In [0]:
user = str(dbutils.fs.ls("file:/Workspace/Users/")).split('/')[3]
filespath = "file:/Workspace/Users/" + f"{user}/treinamento_formula1/landing"
savepath = "dbfs:/mnt/formula1/bronze/circuit"

circuitocsv = spark.read.csv(filespath+"/circuito.csv", header=True)

print('Objetivo: Corrigir os tipos de dados das colunas, inferschema desabilitado para o propósito da ATV')
print('\nAlterando colunas....')
circuitocsv = circuitocsv.withColumn("circuitId", circuitocsv["circuitId"].cast("integer")) \
                         .withColumn("circuitref", circuitocsv["circuitref"].cast("string")) \
                         .withColumn("name", circuitocsv["name"].cast("string")) \
                         .withColumn("location", circuitocsv["location"].cast("string")) \
                         .withColumn("country", circuitocsv["country"].cast("string")) \
                         .withColumn("lat", circuitocsv["lat"].cast("double")) \
                         .withColumn("lng", circuitocsv["lng"].cast("double")) \
                         .withColumn("alt", circuitocsv["alt"].cast("integer")) \
                         .withColumn("url", circuitocsv["url"].cast("string")) 
circuitocsv.printSchema()

print('Criando pasta no dbfs...')
dbutils.fs.mkdirs(savepath)

if(str(dbutils.fs.ls(savepath)).find('circuitoRaw.csv') == -1):
    print('Criando tabela...')
    circuitocsv.write.format('csv').save(f'{savepath}/circuitoRaw.csv', header=True)
else:
    print('Arquivo já existe, sobrescrevendo...')
    circuitocsv.write.format('csv').mode('overwrite').save(f'{savepath}/circuitoRaw.csv', header=True)